In [7]:
import xarray as xr
import numpy as np
import pandas as pd
import math
import dask
NCFILE   = "globalARcatalog_ERA5_1940-2024_v4.0.nc"
_ds      = xr.open_dataset(NCFILE,  engine="netcdf4")
_shapemap = _ds["shapemap"].squeeze(drop=True)  

def ar_near(lat, lon, local_time_str, radius_km=50):
    """
    Check ±6H × ±radius_km if any pixel saw an AR.
    local_time_str in 'YYYY-MM-DD HH:MM' IST.
    """

    
    ist = pd.to_datetime(local_time_str).tz_localize("Asia/Kolkata")
    utc = ist.tz_convert("UTC")

  
    hours = utc.floor("6h")
    times = [hours, hours + pd.Timedelta(hours=6)]

    
    deg_lat = radius_km / 111.0
    deg_lon = radius_km / (111.0 * math.cos(math.radians(lat)))

   
    for t in times:
        
        t6 = t.strftime("%Y-%m-%dT%H:00:00")
        try:
            sub = _shapemap.sel(time=t6, method="nearest")
        except KeyError:
            continue

        
        lat0, lat1 = lat - deg_lat, lat + deg_lat
        lon0, lon1 = (lon % 360) - deg_lon, (lon % 360) + deg_lon

        box = sub.sel(
            lat=slice(lat1, lat0),         
            lon=slice(lon0 % 360, lon1 % 360)
        )

        arr = box.data
        if np.isfinite(arr).any():
            return True, t6

    return False, None


found, when = ar_near(54.50    , -2.90   , "2015-12-05  06:00", radius_km=50)
if found:
    print(f"✅ AR detected near Chennai at ~{when} UTC")
else:
    print("❌ No AR found in that window.")

✅ AR detected near Chennai at ~2015-12-05T00:00:00 UTC


In [25]:
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Load shapefile for Indian states
states_gdf = gpd.read_file("india_states.shp", layer="gadm41_IND_1")  # Make sure the CRS is in degrees (EPSG:4326)
states_gdf = states_gdf.to_crs("EPSG:4326")

# Load dataset
NCFILE   = "globalARcatalog_ERA5_1940-2024_v4.0.nc"
_ds      = xr.open_dataset(NCFILE, engine="netcdf4")
_shapemap = _ds["shapemap"].squeeze(drop=True)

def ar_events_statewise_with_duration(start_date, end_date):
    """
    Detect AR events in India for the given date range.
    Return dictionary of state: list of event times (rounded to 6-hourly bins).
    """

    # Bounding box for India (approximate)
    lat_min, lat_max = 6.0, 37.0
    lon_min, lon_max = 68.0, 97.5
    lon_min360, lon_max360 = lon_min % 360, lon_max % 360

    state_events = {}

    for t in _shapemap.time.values:
        dt = pd.to_datetime(str(t))
        if not (start_date <= dt <= end_date):
            continue

        sub = _shapemap.sel(time=t)
        box = sub.sel(
            lat=slice(lat_max, lat_min),
            lon=slice(lon_min360, lon_max360)
        )

        arr = box.data
        if not np.isfinite(arr).any():
            continue

        lat_vals = box.lat.values
        lon_vals = box.lon.values
        mask = np.isfinite(arr)

        for i, j in zip(*np.where(mask)):
            lat = float(lat_vals[i])
            lon = float(lon_vals[j])
            lon_deg = lon if lon <= 180 else lon - 360  # convert from 0–360 to -180–180

            point = Point(lon_deg, lat)
            match = states_gdf[states_gdf.contains(point)]

            if not match.empty:
                state_name = match.iloc[0]["NAME_1"]  # or your state name column
                if state_name not in state_events:
                    state_events[state_name] = []
                state_events[state_name].append(dt)

    return state_events

def compute_event_durations(state_events):
    """
    From list of timestamps per state, compute grouped durations (6–72 hrs).
    Returns dictionary of state -> list of (start_time, duration_hours)
    """
    duration_summary = {}
    for state, times in state_events.items():
        times_sorted = sorted(pd.to_datetime(times))
        durations = []
    
        current_start = times_sorted[0]
        prev = current_start
        hours = 6
    
        for t in times_sorted[1:]:
            delta = (t - prev).total_seconds() / 3600
    
            if delta == 6:
                hours += 6
            else:
                if hours >= 12:  # ✅ Minimum duration: 24 hrs
                    durations.append((current_start, hours))
                current_start = t
                hours = 6
            prev = t
    
        # Check final group
        if hours >= 12:
            durations.append((current_start, hours))
    
        duration_summary[state] = durations

    return duration_summary  

    print("✔ Finished detecting statewise AR events.")
    print("States found with ARs:", list(statewise_events.keys()))
    print("✔ Finished computing durations.")


# Parameters
start_date = pd.to_datetime("2013-06-10")
end_date = pd.to_datetime("2013-06-16 23:59")

# Detect AR events
statewise_events = ar_events_statewise_with_duration(start_date, end_date)
statewise_durations = compute_event_durations(statewise_events)

# Print summary
for state, events in statewise_durations.items():
    print(f"🗺️ {state}: {len(events)} AR duration events")
    for (start, dur) in events:
        if dur <= 72:
            print(f"   - Start: {start.strftime('%Y-%m-%d %H:%M')} | Duration: {dur} hrs")


🗺️ Gujarat: 1 AR duration events
   - Start: 2013-06-10 06:00 | Duration: 12 hrs
🗺️ Maharashtra: 5 AR duration events
   - Start: 2013-06-10 06:00 | Duration: 12 hrs
   - Start: 2013-06-15 18:00 | Duration: 12 hrs
   - Start: 2013-06-16 00:00 | Duration: 12 hrs
   - Start: 2013-06-16 06:00 | Duration: 12 hrs
   - Start: 2013-06-16 12:00 | Duration: 12 hrs
🗺️ Dadra and Nagar Haveli: 1 AR duration events
   - Start: 2013-06-10 06:00 | Duration: 12 hrs
🗺️ Karnataka: 5 AR duration events
   - Start: 2013-06-10 06:00 | Duration: 12 hrs
   - Start: 2013-06-15 18:00 | Duration: 12 hrs
   - Start: 2013-06-16 00:00 | Duration: 12 hrs
   - Start: 2013-06-16 06:00 | Duration: 12 hrs
   - Start: 2013-06-16 12:00 | Duration: 12 hrs
🗺️ Telangana: 5 AR duration events
   - Start: 2013-06-10 06:00 | Duration: 12 hrs
   - Start: 2013-06-15 18:00 | Duration: 12 hrs
   - Start: 2013-06-16 00:00 | Duration: 12 hrs
   - Start: 2013-06-16 06:00 | Duration: 12 hrs
   - Start: 2013-06-16 12:00 | Duration: 12 

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

import folium
from folium.plugins import MarkerCluster

# Load Indian states shapefile
states_gdf = gpd.read_file("india_states.shp", layer="gadm41_IND_1")
states_gdf = states_gdf.to_crs("EPSG:4326")

# Load dataset
NCFILE = "globalARcatalog_ERA5_1940-2024_v4.0.nc"
_ds = xr.open_dataset(NCFILE, engine="netcdf4")
_shapemap = _ds["shapemap"].squeeze(drop=True)

def ar_events_statewise_with_duration(start_date, end_date):
    """Detect AR events in India for given date range and return state-wise 6-hourly timestamps."""
    lat_min, lat_max = 6.0, 37.0
    lon_min, lon_max = 68.0, 97.5
    lon_min360, lon_max360 = lon_min % 360, lon_max % 360

    state_events = {}

    for t in _shapemap.time.values:
        dt = pd.to_datetime(str(t))
        if not (start_date <= dt <= end_date):
            continue

        sub = _shapemap.sel(time=t)
        box = sub.sel(
            lat=slice(lat_max, lat_min),
            lon=slice(lon_min360, lon_max360)
        )

        arr = box.data
        if not np.isfinite(arr).any():
            continue

        lat_vals = box.lat.values
        lon_vals = box.lon.values
        mask = np.isfinite(arr)

        for i, j in zip(*np.where(mask)):
            lat = float(lat_vals[i])
            lon = float(lon_vals[j])
            lon_deg = lon if lon <= 180 else lon - 360

            point = Point(lon_deg, lat)
            match = states_gdf[states_gdf.contains(point)]

            if not match.empty:
                state_name = match.iloc[0]["NAME_1"]
                if state_name not in state_events:
                    state_events[state_name] = []
                state_events[state_name].append(dt)

    return state_events

def compute_event_durations(state_events):
    """Compute state-wise AR event durations filtered by AR criteria (12–72 hrs)."""
    duration_summary = {}

    for state, times in state_events.items():
        times_sorted = sorted(pd.to_datetime(times))
        if not times_sorted:
            continue

        durations = []
        current_start = times_sorted[0]
        prev = current_start
        hours = 6

        for t in times_sorted[1:]:
            delta = (t - prev).total_seconds() / 3600

            if delta == 6:
                hours += 6
            else:
                if 12 <= hours <= 72:
                    durations.append((current_start, hours))
                current_start = t
                hours = 6
            prev = t

        # Final group
        if 12 <= hours <= 72:
            durations.append((current_start, hours))

        if durations:
            duration_summary[state] = durations

    return duration_summary

# Parameters
start_date = pd.to_datetime("2013-06-10")
end_date = pd.to_datetime("2013-06-16 23:59")

# Detect and compute
statewise_events = ar_events_statewise_with_duration(start_date, end_date)
statewise_durations = compute_event_durations(statewise_events)

# Summary
print(f"🌀 AR Events from {start_date.date()} to {end_date.date()}")
for state, events in statewise_durations.items():
    print(f"🗺️ {state}: {len(events)} AR duration events")
    for (start, dur) in events:
        print(f"   - Start: {start.strftime('%Y-%m-%d %H:%M')} | Duration: {dur} hrs")



# Create base map centered on India
m = folium.Map(location=[22.0, 80.0], zoom_start=5, tiles="CartoDB positron")

# Cluster for grouping nearby points
marker_cluster = MarkerCluster().add_to(m)

# Collect AR events by pixel (each: lat, lon, timestamp, duration)
ar_pixel_events = []

for state, durations in statewise_durations.items():
    for start_time, duration in durations:
        if 12 <= duration <= 72:
            event_times = pd.date_range(start=start_time, periods=int(duration / 6), freq="6h")
            for t in event_times:
                if t not in statewise_events.get(state, []):
                    continue

                sub = _shapemap.sel(time=t)
                box = sub.sel(lat=slice(37.0, 6.0), lon=slice(68.0 % 360, 97.5 % 360))
                arr = box.data
                if not np.isfinite(arr).any():
                    continue

                lat_vals = box.lat.values
                lon_vals = box.lon.values
                mask = np.isfinite(arr)

                for i, j in zip(*np.where(mask)):
                    lat = float(lat_vals[i])
                    lon = float(lon_vals[j])
                    lon_deg = lon if lon <= 180 else lon - 360
                    ar_pixel_events.append((lat, lon_deg, t.strftime("%Y-%m-%d %H:%M"), duration))

# Add events to map
for lat, lon, timestamp, duration in ar_pixel_events:
    folium.CircleMarker(
        location=[lat, lon],
        radius=4,
        popup=f"<b>Time:</b> {timestamp}<br><b>Duration:</b> {duration} hrs",
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.5
    ).add_to(marker_cluster)

# Save to HTML file and open in browser
m.save("ar_events_interactive_map.html")
print('map saved')


🌀 AR Events from 2013-06-10 to 2013-06-16
🗺️ Gujarat: 1 AR duration events
   - Start: 2013-06-10 06:00 | Duration: 12 hrs
🗺️ Maharashtra: 5 AR duration events
   - Start: 2013-06-10 06:00 | Duration: 12 hrs
   - Start: 2013-06-15 18:00 | Duration: 12 hrs
   - Start: 2013-06-16 00:00 | Duration: 12 hrs
   - Start: 2013-06-16 06:00 | Duration: 12 hrs
   - Start: 2013-06-16 12:00 | Duration: 12 hrs
🗺️ Dadra and Nagar Haveli: 1 AR duration events
   - Start: 2013-06-10 06:00 | Duration: 12 hrs
🗺️ Karnataka: 5 AR duration events
   - Start: 2013-06-10 06:00 | Duration: 12 hrs
   - Start: 2013-06-15 18:00 | Duration: 12 hrs
   - Start: 2013-06-16 00:00 | Duration: 12 hrs
   - Start: 2013-06-16 06:00 | Duration: 12 hrs
   - Start: 2013-06-16 12:00 | Duration: 12 hrs
🗺️ Telangana: 5 AR duration events
   - Start: 2013-06-10 06:00 | Duration: 12 hrs
   - Start: 2013-06-15 18:00 | Duration: 12 hrs
   - Start: 2013-06-16 00:00 | Duration: 12 hrs
   - Start: 2013-06-16 06:00 | Duration: 12 hrs
  

In [ ]:
# Define bounding boxes for regions (approximate)
regions = {
    "North": {"lat_min": 25, "lat_max": 37, "lon_min": 68, "lon_max": 90},
    "South": {"lat_min": 6, "lat_max": 21, "lon_min": 68, "lon_max": 90},
    "East":  {"lat_min": 20, "lat_max": 31, "lon_min": 86, "lon_max": 97.5},
    "West":  {"lat_min": 20, "lat_max": 30, "lon_min": 68, "lon_max": 78},
}

# Organize events by region
region_events = {r: [] for r in regions}

for lat, lon, timestamp, duration in ar_pixel_events:
    for region, bounds in regions.items():
        if bounds["lat_min"] <= lat <= bounds["lat_max"] and bounds["lon_min"] <= lon <= bounds["lon_max"]:
            region_events[region].append((lat, lon, timestamp, duration))
            break

# Generate separate maps
for region, events in region_events.items():
    print(f"🔹 Saving map for {region} region with {len(events)} events")

    m = folium.Map(location=[
        (regions[region]["lat_min"] + regions[region]["lat_max"]) / 2,
        (regions[region]["lon_min"] + regions[region]["lon_max"]) / 2
    ], zoom_start=5, tiles="CartoDB positron")

    marker_cluster = MarkerCluster().add_to(m)

    for lat, lon, timestamp, duration in events:
        folium.CircleMarker(
            location=[lat, lon],
            radius=4,
            popup=f"<b>Time:</b> {timestamp}<br><b>Duration:</b> {duration} hrs",
            color="blue",
            fill=True,
            fill_color="blue",
            fill_opacity=0.5
        ).add_to(marker_cluster)

    m.save(f"ar_events_{region.lower()}_india.html")
    print(f"✅ Saved: ar_events_{region.lower()}_india.html")


🔹 Saving map for North region with 218076 events


In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Load Indian states shapefile
states_gdf = gpd.read_file("india_states.shp", layer="gadm41_IND_1")
states_gdf = states_gdf.to_crs("EPSG:4326")

# Load dataset
NCFILE = "globalARcatalog_ERA5_1940-2024_v4.0.nc"
_ds = xr.open_dataset(NCFILE, engine="netcdf4")
_shapemap = _ds["shapemap"].squeeze(drop=True)

def ar_events_statewise_with_duration(start_date, end_date):
    lat_min, lat_max = 6.0, 37.0
    lon_min, lon_max = 68.0, 97.5
    lon_min360, lon_max360 = lon_min % 360, lon_max % 360

    state_events = {}
    for t in _shapemap.time.values:
        dt = pd.to_datetime(str(t))
        if not (start_date <= dt <= end_date):
            continue
        if dt.month not in [6, 7, 8]:  # Only June–August
            continue

        sub = _shapemap.sel(time=t)
        box = sub.sel(lat=slice(lat_max, lat_min), lon=slice(lon_min360, lon_max360))
        arr = box.data
        if not np.isfinite(arr).any():
            continue

        lat_vals = box.lat.values
        lon_vals = box.lon.values
        mask = np.isfinite(arr)

        for i, j in zip(*np.where(mask)):
            lat = float(lat_vals[i])
            lon = float(lon_vals[j])
            lon_deg = lon if lon <= 180 else lon - 360
            point = Point(lon_deg, lat)
            match = states_gdf[states_gdf.contains(point)]

            if not match.empty:
                state_name = match.iloc[0]["NAME_1"]
                if state_name not in state_events:
                    state_events[state_name] = []
                state_events[state_name].append(dt)

    return state_events

def compute_event_durations(state_events):
    duration_summary = {}
    for state, times in state_events.items():
        times_sorted = sorted(pd.to_datetime(times))
        if not times_sorted:
            continue

        durations = []
        current_start = times_sorted[0]
        prev = current_start
        hours = 6

        for t in times_sorted[1:]:
            delta = (t - prev).total_seconds() / 3600
            if delta == 6:
                hours += 6
            else:
                if 12 <= hours <= 72:
                    durations.append((current_start, hours))
                current_start = t
                hours = 6
            prev = t

        if 12 <= hours <= 72:
            durations.append((current_start, hours))

        if durations:
            duration_summary[state] = durations

    return duration_summary

# Run detection for 2024 (June–August)
start_date = pd.to_datetime("2024-06-01")
end_date = pd.to_datetime("2024-08-31 23:59")

statewise_events = ar_events_statewise_with_duration(start_date, end_date)
statewise_durations = compute_event_durations(statewise_events)

# Count total AR events across India
total_ar_events = sum(len(events) for events in statewise_durations.values())
print("✅ Total AR events in India (June–August 2024):", total_ar_events)
